## Model and Evaluation 3

In [21]:
import pandas as pd
import numpy as np
import seaborn as sns
import yellowbrick as yb

player = pd.read_csv('~/7331_Lab2/data/player.csv')
playerNoSal = pd.read_csv('~/7331_Lab2/data/playerNoSal.csv')
playerScale = pd.read_csv('~/7331_Lab2/data/playerscale.csv')
playerNoSalScale = pd.read_csv('~/7331_Lab2/data/playerNoSalscale.csv')

In [22]:
player = player.drop(['Unnamed: 0'], axis=1)
playerNoSal = playerNoSal.drop(['Unnamed: 0'], axis=1)
playerScale = playerScale.drop(['Unnamed: 0'], axis=1)
playerNoSalScale = playerNoSalScale.drop(['Unnamed: 0'], axis=1)

print(player.info())
print(playerNoSal.info())
print(playerScale.info())
print(playerNoSalScale.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19820 entries, 0 to 19819
Data columns (total 43 columns):
yearID                 19820 non-null int64
playerID               19820 non-null object
G_x                    19820 non-null int64
AB                     19820 non-null int64
R                      19820 non-null int64
H                      19820 non-null int64
2B                     19820 non-null int64
3B                     19820 non-null int64
HR                     19820 non-null int64
RBI                    19820 non-null float64
SB_x                   19820 non-null float64
CS_x                   19820 non-null float64
BB                     19820 non-null int64
SO                     19820 non-null float64
IBB                    19820 non-null float64
HBP                    19820 non-null float64
SH                     19820 non-null float64
SF                     19820 non-null float64
GIDP                   19820 non-null float64
stint                  19820 non-nul

In [5]:
### Experiment with Subsetting Data
#Games

### Linear Discriminate Analysis

Variables to vary:  
- solver = lsqr, eigen 
- n_components = 4:30:2
- shrinkage = none, auto 
- priors = priors, no priors

StratifiedKFold Cross Validation due to number large number of outfielders compared to other positions.

#### LDA Priors

In [48]:
df = player[['yearID', 'playerID', 'Primary']]
df = df.groupby(['Primary']).count().reset_index()
df['Pct'] = (df.playerID / len(player))
df = df.drop(['yearID', 'playerID'], axis=1)
print(df)

myPriors = np.array(df.Pct)
print(myPriors)

#print(playerScale.index)
#print(playerScale)

  Primary       Pct
0      1B  0.120535
1      2B  0.116246
2      3B  0.112513
3       C  0.161857
4      DH  0.014329
5      OF  0.373764
6      SS  0.100757
[0.12053481 0.11624622 0.11251261 0.16185671 0.01432896 0.37376387
 0.10075681]


In [51]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import StratifiedKFold, ShuffleSplit, cross_validate, GridSearchCV
from sklearn import metrics as mt

#Create Lists to Automate LDA Model Search
mySolv = ['lsqr', 'eigen']
myNComponents = range(4,30,2)
myShrink = ['None', 'auto']

#Create Target and Data sets
X = playerScale.drop(['Primary', 'playerID'], axis=1)
#X = X.reset_index(drop=True)
y = playerScale['Primary']
#y = y.reset_index(drop=True)
yhat = np.zeros(y.shape)

print(X.index, y.index)
print(X)
print(y)

#Create cross validation interator
cv = StratifiedKFold(n_splits=10)

#Create LDA Object
clf = LinearDiscriminantAnalysis(n_components=None, priors=myPriors, shrinkage='auto', solver='lsqr')
clf.fit(X, y)
print("1st Pass Accuracy:", clf.score(X, y))

for train, test in cv.split(X, y):
    clf.fit(X[train],y[train])
    yhat[test] = clf.predict(X[test])
    
total_accuracy = mt.accuracy_score(y, yhat)
print ('LDA accuracy', total_accuracy)

RangeIndex(start=0, stop=19820, step=1) RangeIndex(start=0, stop=19820, step=1)
            G_x        AB         R         H        2B        3B        HR  \
0     -0.509599 -0.662679 -0.716804 -0.768817 -0.840291 -0.216968 -0.592251   
1      0.206484 -0.442943 -0.238661 -0.369206 -0.504161  0.722066 -0.168550   
2     -1.322450 -1.167072 -1.035566 -1.098931 -1.008355 -0.686485 -0.698177   
3     -1.283743 -1.097156 -1.035566 -1.098931 -1.008355 -0.686485 -0.804102   
4      0.148423 -0.048418 -0.047404 -0.021718  0.336164 -0.686485 -0.168550   
5     -0.064466 -0.163280 -0.334290 -0.403955 -0.588193 -0.686485 -0.380401   
6      0.438727  0.655735  0.494491  0.603761  0.336164  1.661100  1.632178   
7     -0.296710 -0.323088 -0.461794 -0.247585 -0.420128 -0.686485  0.043300   
8     -0.664428 -0.887409 -0.780556 -0.838315 -0.504161 -0.686485 -0.592251   
9      1.251578  1.329923  1.355148  1.298737  0.924391  1.661100 -0.698177   
10     0.651617  0.391053  0.398862  0.360519  0.08

KeyError: '[ 1382  1391  1429 ... 19817 19818 19819] not in index'